# Capture

## Pseudo 16 bit recording

This script will record images form a pylon compatible 10-bit camera. At regular interval defined by `fpm` (frames par minute) it will collect 64 images and sum them to form one 16-bit image. The final effective resolution should be 13 bits according to the *oversampling-decimation* principle.

Then the image is save in `folder`. The total number of frames to take is defined by `num_frames`.

It is usefull to use Pylon software first to adjust the camera settings, and then close Pylon.

In [1]:
import numpy as np
import sched, time
from datetime import date

from skimage.io import imsave
from skimage.util import img_as_uint
from skimage import exposure

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interactive, IntRangeSlider, IntSlider

from pypylon import pylon

In [2]:
# Folder (relative path from this script)
folder = 'images'

# Frame par minute
fpm = 10

# Number of frames to record
num_frames = 10

# Oversampling index (will average over 2**n_oversampling shots)
# Recommanded value: 6 (will sum 64 images and the result in a 16-bit array)
# Recommanded maximum: 12 (will average 4096 images, resulting in pseudo 16-bit image)
n_oversampling = 6

## Open camera

In [3]:
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [4]:
def grap_pseudo16bit(n_oversampling):
    """Sum 2**n_oversampling images from the 10-bit camera and rescqle to obtain one pseudo-16-bit image."""
    img32 = np.zeros((camera.Height.GetValue(), camera.Width.GetValue()), dtype=np.uint32)
    
    # Take images
    camera.StartGrabbingMax(2**n_oversampling)
    
    while camera.IsGrabbing():
        grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

        if grabResult.GrabSucceeded():
            # Access the image data.
            img32 += grabResult.Array

        grabResult.Release()
    # Rescale to uint16
    if n_oversampling < 6:
        # multiply by 2**(6-n_oversamp)
        img16 = np.left_shift(img32, 6-n_oversampling).astype(np.uint16)
    elif n_oversampling > 6:
        # devide by 2**(n_oversamp-6)
        img16 = np.right_shift(img32, n_oversampling-6).astype(np.uint16)
    else:
        # Just cast to 16 bits
        img16 = img32.astype(np.uint16)
    
    return img16

## Adjust settings

In [6]:
# Initial value
init_value = int(camera.ExposureTime.GetValue())
# Emulation
#init_value = 10_000

def adjust(exposure):
    # Set new exposure
    camera.ExposureTime.SetValue(float(exposure))
    
    # Get image
    img = grap_pseudo16bit(n_oversampling)
    
    # Emulation
    #img = np.random.randint(0,1023,size=(1000,1000), dtype=np.uint16)
    
    plt.rcParams["figure.figsize"] = (15,15)
    cmap = cm.get_cmap("gray").copy()
    cmap.set_over('red')
    
    figure, ax = plt.subplots(nrows=2)
    
    ax[0].imshow(img, cmap=cmap, vmin=0, vmax=2**16-2**6-1)
    
    ax[1].hist(img.flatten(), bins=256, log=True)
    plt.show()
    
    result = {'exposure': exposure,
             'width': 0,
             'height': 0}
    
    return result
    
w = interactive(adjust, exposure=IntSlider(value=init_value,min=5000,max=12000, continuous_update=False))
w

interactive(children=(IntSlider(value=8000, continuous_update=False, description='exposure', max=12000, min=50…

## Capture frames

In [7]:
def capture(frame_number=0):
    img = grap_pseudo16bit(n_oversampling)
    file_name = f'{folder}/{date.today().isoformat()}_{frame_number:04d}.tiff'
    imsave(file_name, img)
    
    print(f'Frame number {frame_number}')

In [8]:
# Schedule capture and run immediately
s = sched.scheduler(time.time, time.sleep)

for i in range(num_frames):
    s.enter(i*60./fpm, 1, capture, kwargs={'frame_number':i})
    
s.run()

Frame number 0
Frame number 1
Frame number 2
Frame number 3
Frame number 4
Frame number 5
Frame number 6
Frame number 7
Frame number 8
Frame number 9


In [9]:
np.sqrt(4*60e-6/np.pi/0.11)

0.02635325970593146

In [10]:
0.3*60

18.0